# Descrição do Projeto

A empresa Sweet Lift Taxi coletou dados históricos sobre pedidos de táxi nos aeroportos. Para atrair mais motoristas durante o horário de pico, precisamos prever a quantidade de pedidos de táxi para a próxima hora. 	
Construa um modelo para tal predição.

A métrica REQM no conjunto de teste não deve ser superior a 48.

## Instruções do projeto

1. Faça download dos dados e faça uma nova amostragem em uma hora.
2. Analise os dados
3. Treine diferentes modelos com diferentes hiperparâmetros. A amostra de teste deve ser 10% do conjunto de dados inicial.
4. Teste os dados usando a amostra de teste e forneça uma conclusão.

## Descrição dos dados

Os dados são armazenados no arquivo `taxi.csv`. O número de pedidos está na coluna `num_orders`.

## Preparação 

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from lightgbm import LGBMRegressor


data = pd.read_csv(
    '/datasets/taxi.csv', index_col=[0], parse_dates=[0]
)
data.sort_index(inplace=True)
data = data.resample('1H').sum()

## Análise

In [2]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4416 entries, 2018-03-01 00:00:00 to 2018-08-31 23:00:00
Freq: H
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   num_orders  4416 non-null   int64
dtypes: int64(1)
memory usage: 69.0 KB
None


In [3]:
def make_features(data, max_lag, rolling_mean_size):
    data['year'] = data.index.year
    data['month'] = data.index.month
    data['day'] = data.index.day
    data['dayofweek'] = data.index.dayofweek
    data['hour'] = data.index.hour

    for lag in range(1, max_lag + 1):
        data['lag_{}'.format(lag)] = data['num_orders'].shift(lag)

    data['rolling_mean'] = (
        data['num_orders'].shift().rolling(rolling_mean_size).mean()
    )


make_features(data, 4, 10)
print(data.head())

                     num_orders  year  month  day  dayofweek  hour  lag_1  \
datetime                                                                    
2018-03-01 00:00:00         124  2018      3    1          3     0    NaN   
2018-03-01 01:00:00          85  2018      3    1          3     1  124.0   
2018-03-01 02:00:00          71  2018      3    1          3     2   85.0   
2018-03-01 03:00:00          66  2018      3    1          3     3   71.0   
2018-03-01 04:00:00          43  2018      3    1          3     4   66.0   

                     lag_2  lag_3  lag_4  rolling_mean  
datetime                                                
2018-03-01 00:00:00    NaN    NaN    NaN           NaN  
2018-03-01 01:00:00    NaN    NaN    NaN           NaN  
2018-03-01 02:00:00  124.0    NaN    NaN           NaN  
2018-03-01 03:00:00   85.0  124.0    NaN           NaN  
2018-03-01 04:00:00   71.0   85.0  124.0           NaN  


## Treinamento

In [4]:
train, test = train_test_split(data, shuffle=False, test_size=0.1)
train = train.dropna()
test = test.dropna()

features_train = train.drop(columns='num_orders')
target_train = train['num_orders']

features_test = test.drop(columns='num_orders')
target_test = test['num_orders']

In [5]:
print(train.head())
print(test.head())

                     num_orders  year  month  day  dayofweek  hour  lag_1  \
datetime                                                                    
2018-03-01 10:00:00          64  2018      3    1          3    10   69.0   
2018-03-01 11:00:00          96  2018      3    1          3    11   64.0   
2018-03-01 12:00:00          30  2018      3    1          3    12   96.0   
2018-03-01 13:00:00          32  2018      3    1          3    13   30.0   
2018-03-01 14:00:00          48  2018      3    1          3    14   32.0   

                     lag_2  lag_3  lag_4  rolling_mean  
datetime                                                
2018-03-01 10:00:00   34.0   15.0   12.0          52.5  
2018-03-01 11:00:00   69.0   34.0   15.0          46.5  
2018-03-01 12:00:00   64.0   69.0   34.0          47.6  
2018-03-01 13:00:00   96.0   64.0   69.0          43.5  
2018-03-01 14:00:00   30.0   96.0   64.0          40.1  
                     num_orders  year  month  day  dayofweek 

In [6]:
 param_grids = {
        "linearregression": [
            {"fit_intercept": True},
        ],
        "randomforestregressor": [
            {"n_estimators": 50, "max_depth": 10, "n_jobs": -1},
            {"n_estimators": 100, "max_depth": 15, "n_jobs": -1},
            {"n_estimators": 100, "max_depth": 8, "n_jobs": -1},
        ],
        "lgbmregressor": [
            {
                "n_estimators": 50,
                "learning_rate": 0.1,
                "max_depth": 8,
                "num_leaves": 31,
                "n_jobs": -1,
                "verbosity": -1,
            },
            {
                "n_estimators": 100,
                "learning_rate": 0.1,
                "max_depth": 10,
                "num_leaves": 50,
                "n_jobs": -1,
                "verbosity": -1,
            },
            {
                "n_estimators": 80,
                "learning_rate": 0.15,
                "max_depth": 6,
                "num_leaves": 64,
                "n_jobs": -1,
                "verbosity": -1,
            },
        ],
    }


## Testando

In [7]:
best_params = None
best_mse = float("inf")

for params in param_grids["linearregression"]:
    model = LinearRegression(**params)
    model.fit(features_train, target_train)
    train_predictions = model.predict(features_train)
    train_mse = mean_squared_error(target_train, train_predictions)
    
    if train_mse < best_mse:
        best_mse = train_mse
        best_params = params
        model = LinearRegression(**best_params)
        model.fit(features_train, target_train)

print(f"Melhores Hiperparâmetros: {best_params}")
print(f"Melhor REQM no Conjunto de Treino: {np.sqrt(best_mse)}")
    

Melhores Hiperparâmetros: {'fit_intercept': True}
Melhor REQM no Conjunto de Treino: 30.493054556811586


In [9]:
test_predictions = model.predict(features_test)
test_mse = mean_squared_error(target_test, test_predictions)
r2 = r2_score(target_test, test_predictions)

print(f"REQM no Conjunto de Teste: {np.sqrt(test_mse)}")
print(f"R2 no Conjunto de Teste: {r2}")


REQM no Conjunto de Teste: 53.11676956083512
R2 no Conjunto de Teste: 0.17697068347124945


*Observações: REQM acima do objetivo de 48. R2 baixo*

In [10]:
best_params = None
best_mse = float("inf")

for params in param_grids["lgbmregressor"]:
    model = LGBMRegressor(**params)
    model.fit(features_train, target_train)
    train_predictions = model.predict(features_train)
    train_mse = mean_squared_error(target_train, train_predictions)
    
    if train_mse < best_mse:
        best_mse = train_mse
        best_params = params
        model = LGBMRegressor(**best_params)
        model.fit(features_train, target_train)

print(f"Melhores Hiperparâmetros: {best_params}")
print(f"Melhor REQM no Conjunto de Treino: {np.sqrt(best_mse)}")

Melhores Hiperparâmetros: {'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 10, 'num_leaves': 50, 'n_jobs': -1, 'verbosity': -1}
Melhor REQM no Conjunto de Treino: 13.731508700300804


In [11]:
test_predictions = model.predict(features_test)
test_mse = mean_squared_error(target_test, test_predictions)
r2 = r2_score(target_test, test_predictions)

print(f"REQM no Conjunto de Teste: {np.sqrt(test_mse)}")
print(f"R2 no Conjunto de Teste: {r2}")

REQM no Conjunto de Teste: 43.19216038420405
R2 no Conjunto de Teste: 0.4557958483008908


*Observações: REQM abaixo do objetivo de 48. R2 médio*

In [12]:
best_params = None
best_mse = float("inf")

for params in param_grids["randomforestregressor"]:
    model = RandomForestRegressor(**params)
    model.fit(features_train, target_train)
    train_predictions = model.predict(features_train)
    train_mse = mean_squared_error(target_train, train_predictions)
    
    if train_mse < best_mse:
        best_mse = train_mse
        best_params = params
        model = RandomForestRegressor(**best_params)
        model.fit(features_train, target_train)

print(f"Melhores Hiperparâmetros: {best_params}")
print(f"Melhor REQM no Conjunto de Treino: {np.sqrt(best_mse)}")


Melhores Hiperparâmetros: {'n_estimators': 100, 'max_depth': 15, 'n_jobs': -1}
Melhor REQM no Conjunto de Treino: 9.311761244658104


In [13]:
test_predictions = model.predict(features_test)
test_mse = mean_squared_error(target_test, test_predictions)
r2 = r2_score(target_test, test_predictions)

print(f"REQM no Conjunto de Teste: {np.sqrt(test_mse)}")
print(f"R2 no Conjunto de Teste: {r2}")

REQM no Conjunto de Teste: 49.28550418920476
R2 no Conjunto de Teste: 0.29141752999864323


*Observações: REQM acima do objetivo de 48. R2 baixo*

### Conclusão
- O único modelo que consegue um REQM abaixo de 48 no conjunto de teste é o de GradientBoosting, LGBMRegressor, com os parâmetros: `n_estimators`: 100, `learning_rate`: 0.1, `max_depth`: 10, `num_leaves`: 50, `n_jobs`: -1, `verbosity`: -1

# Revisão da checklist

- [x]  O Jupyter Notebook está aberto.

- [ ]  O código está livre de erros
- [ ]  As células com o código foram organizadas em ordem de execução.
- [ ]  Os dados foram baixados e preparados

- [ ]  Os dados foram analisados
- [ ]  O modelo foi treinado e os hiperparâmetros foram selecionados
- [ ]  	
O modelo foi avaliado. Uma conclusão foi fornecida

- [ ] 
O REQM para o conjunto de teste não é maior que 48